In [15]:
import vcf # type: ignore
import random
def str_vcf_to_genepop(vcf_file, output_file):
    """Convert VCF format to GenePop format."""
    vcf_reader = vcf.Reader(filename=vcf_file)
    with open(output_file, 'w') as genepop:
        # Write title
        genepop.write("STRvcf2genepop\n")
        
        # Collect locus names and allele count per locus
        locus_names = []
        unique_ids = set()
        locus_allele_count = {}
        pos_to_locus_id = {}  # Mapping from record.POS to unique locus_id
        for record in vcf_reader:
            allele_combinations = {}
            ref_repeat_length = len(record.REF)
            for sample in record.samples:
                gb_data = getattr(sample.data, 'GB', None)
                if gb_data:
                    allele_diffs = gb_data.split('/')
                    # Assuming a pair of allele diffs is available for each sample
                    if len(allele_diffs) == 2:
                        alleles = []
                        for diff in allele_diffs:
                            if int(diff) < -100:
                                allele = ref_repeat_length  # Handle extreme diffs
                            else:
                                allele = ref_repeat_length + int(diff)
                            alleles.append(f"{allele:03}")
                        # Combine alleles into a string and count the combination
                        allele_key = ''.join(alleles)
                        allele_combinations[allele_key] = allele_combinations.get(allele_key, 0) + 1
            # Only include loci with more than one allele combination
            if len(allele_combinations) > 1:
                locus_id = f"STR_{record.POS}"
                while locus_id in unique_ids:
                    # Add a random digit to the end of the locus_id if it's not unique
                    locus_id += str(random.randint(0, 9))
                unique_ids.add(locus_id)
                locus_names.append(locus_id)
                locus_allele_count[locus_id] = allele_combinations
                pos_to_locus_id[record.POS] = locus_id
        genepop.write("\n".join(locus_names) + "\n")
         
        # Reset the reader to read samples
        vcf_reader = vcf.Reader(filename=vcf_file)
        
        # Initialize a dictionary to collect genotypes for each sample
        samples_genotypes = {}
        for record in vcf_reader:
            ref_repeat_length = len(record.REF)
            # Use the mapping to access the correct locus_id
            locus_id = pos_to_locus_id.get(record.POS)
            if locus_id and locus_id in locus_allele_count:
                for sample in record.samples:
                    if sample.sample not in samples_genotypes:
                        samples_genotypes[sample.sample] = []
                    gb_data = getattr(sample.data, 'GB', None)
                    genotype = '000000'  # Default for missing data
                    if gb_data:
                        allele_diffs = gb_data.split('/')
                        alleles = []
                        for diff in allele_diffs:
                            if int(diff) < -100:
                                alleles.append(ref_repeat_length)
                            else:
                                adjusted_repeat_length = ref_repeat_length + int(diff)
                                alleles.append(adjusted_repeat_length)
                        genotype = ''.join(f"{allele:03}" for allele in alleles)
                    samples_genotypes[sample.sample].append(genotype)
        
        # # Write sample genotypes to file
        # for sample, genotypes in samples_genotypes.items():
        #     genepop.write(f"{sample}, " + " ".join(genotypes) + "\n")

        # Organize samples by prefix and write genotypes with POP labels
        prefix_groups = {}
        for sample, genotypes in samples_genotypes.items():
            prefix = sample.split('_')[0]
            if prefix not in prefix_groups:
                prefix_groups[prefix] = []
            prefix_groups[prefix].append((sample, genotypes))
        
        for prefix, samples in prefix_groups.items():
            genepop.write("POP\n")
            for sample, genotypes in samples:
                genepop.write(f"{sample}, " + " ".join(genotypes) + "\n")


In [16]:
# Example usage (commented out):
str_vcf_to_genepop("SSR.filted.vcf", "SSR.filted.gen")